## This script plots the final project deliverable for ENERGY 312 at Stanford University Fall 2021

Jimmy Zhang 

06440538

November 12th, 2021



In [88]:
!pip install geopandas
!pip install pyshp
!pip install shapely
!pip install plotly --upgrade
!pip install chart_studio

In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
df_zips = pd.read_csv('https://raw.githubusercontent.com/jz486350/ENERGY-312-SONOMA/main/nodes.csv')
df_zips.head()

,origin,trips,location,lat,long
0,94922,8,Bodega,38.3514,-122.9741
1,94923,610,Bodega Bay,38.3309,-123.0373
2,94927,289,Rohnert Park,38.3396,-122.7011
3,94928,8324,Rohnert Park,38.3470,-122.6941
4,94931,2246,Cotati,38.3259,-122.7048


In [ ]:
df_trips = pd.read_csv('https://raw.githubusercontent.com/jz486350/ENERGY-312-SONOMA/main/trips.csv')
df_trips.head()

,start,end,trips,start_lat,start_long,end_lat,end_long,dist_miles,width
0,95403,95401,2042,38.4822,-122.7473,38.4432,-122.7547,2.724217,6.000000
1,95403,95492,2039,38.4822,-122.7473,38.5443,-122.8073,5.378865,5.988484
2,95403,95404,1957,38.4822,-122.7473,38.4405,-122.7144,3.386666,5.673704
3,94952,94954,1854,38.2403,-122.6777,38.2507,-122.6155,3.450834,5.278311
4,95403,95407,1700,38.4822,-122.7473,38.4089,-122.7339,5.116188,4.687140


## Outline Version

In [ ]:
fig = go.Figure()

trips = []
for i in range(len(df_trips)):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [df_trips['start_long'][i], df_trips['end_long'][i]],
            lat = [df_trips['start_lat'][i], df_trips['end_lat'][i]],
            mode = 'lines',
            line = dict(width = df_trips['width'][i]*0.9 ,color = 'red'),
            opacity = float(( df_trips['trips'][i]) / float(df_trips['trips'].max())*0.9+0.10),
        )
    ) 
    
for i in range(len(df_zips)):
  fig.add_trace(go.Scattergeo(
      locationmode = 'USA-states',
      lon = np.array([df_zips['long'][i]]),
      lat = np.array([df_zips['lat'][i]]),
      hoverinfo = 'text',
      text = 'ZIP: ' + str(df_zips['origin'][i]) + '<br>Location: ' + str(df_zips['location'][i]) + '<br>Trips: ' +str(df_zips['trips'][i]),
      mode = 'markers',
      marker = dict(
          size = 15*(df_zips['trips'][i]/max(df_zips['trips']))+4,
          color = 'rgb(225, 0, 0)',
          line = dict(
              width = 3,
              color = 'rgba(68, 68, 68, 0)'
          )
      )))

fig.update_layout(
    hovermode='closest',
    geo = go.layout.Geo(
        scope = 'north america',
        projection = go.layout.geo.Projection(
            type = 'azimuthal equal area',
            scale=130
        ),
        center={'lat': 38.496, 'lon': -122.864},
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
    autosize=True,
    showlegend = False,
)

fig.show()

## Mapbox Version

In [90]:
fig = go.Figure()

# Dynamic Trip Tracing for #trips > 50
trips = []
for i in range(len(df_trips[:300])):
    fig.add_trace(
        go.Scattermapbox(
          lon = [df_trips['start_long'][i], df_trips['end_long'][i]],
          lat = [df_trips['start_lat'][i], df_trips['end_lat'][i]],
          mode = 'lines',
          line = dict(width = df_trips['width'][i]*0.9 ,color = 'rgb(72,209,204)'),
          opacity = float(( df_trips['trips'][i]) / float(df_trips['trips'].max())*0.9+0.10),
        )
    )

# Aggregate Tracing for #trips < 50
lons = []
lats = []
lons = np.empty(3 * len(df_trips[300:]))
lons[::3] = df_trips[300:]['start_long']
lons[1::3] = df_trips[300:]['end_long']
lons[2::3] = None
lats = np.empty(3 * len(df_trips[300:]))
lats[::3] = df_trips[300:]['start_lat']
lats[1::3] = df_trips[300:]['end_lat']
lats[2::3] = None

fig.add_trace(
    go.Scattermapbox(
        lon = lons,
        lat = lats,
        mode = 'lines',
        line = dict(width = 1,color = 'rgb(72,209,204)'),
        opacity = 0.1,
        visible=False
    )
)

# Overlaying Nodes over Trace
for i in range(len(df_zips)):
  fig.add_trace(
      go.Scattermapbox(
          lon = np.array([df_zips['long'][i]]),
          lat = np.array([df_zips['lat'][i]]),
          hoverinfo = 'text',
          text = 'ZIP: ' + str(df_zips['origin'][i]) + '<br>Location: ' + str(df_zips['location'][i]) + '<br>Trips: ' +str(df_zips['trips'][i]),
          mode = 'markers',
          marker = go.scattermapbox.Marker(
              size=15*(df_zips['trips'][i]/max(df_zips['trips']))+4,
              color='rgb(0,206,209)',
          )
      )
  )

# Buttons for Toggling Traces {Hide/Show #Trips < 50}
buttons = []
button = dict(
    method='restyle',
    label='Hide/Show # Trips < 50',
    visible=True,
    args=[{'visible':True},[300]],
    args2 = [{'visible': False},[300]],
)

# Defining Raster Base, Mapbox Zoom/Width/Height, Button
fig.update_layout(
    
    mapbox_style="stamen-toner",

    title={
        'text': 'Sonoma County Employment-Based Trips<br>(Hover for ZIP, Name, # Trips)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },

    showlegend = False,

    mapbox = dict(
        center=go.layout.mapbox.Center(
            lat=38.496,
            lon=-122.864
        ),
        zoom=8.3,
    ),

    updatemenus=[
        dict(
            type = "buttons",
            direction = "left",
            buttons=list([
                button
            ]),
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.5,
            xanchor='center',
            y=0,
            yanchor='top'
        ),
    ],
    # margin=dict(l=50, r=50, t=100, b=50),
    # autosize = False,
    # width = 800, height = 600

)


fig.show()

In [91]:
fig.write_html('/content/drive/MyDrive/Energy 312/Plot/Sonoma2.html')